# Model v2.1 - Keras Version

In [1]:
import tensorflow as tf

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.externals import joblib

In [5]:
import pandas as pd
import numpy as np
import os
import math

In [6]:
version = 2.1
version_str = "v" + str(version)

### Load dataset

In [7]:
LABEL = "price_doc"

In [8]:
kaggle_test = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/test.csv/test.csv")
macro_df = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/macro.csv/macro.csv")
final_kaggle_df = pd.merge(kaggle_test, macro_df, on='timestamp')

In [9]:
final_kaggle_df[LABEL] = 0.0

In [10]:
final_train_df = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/final_training_dataset.csv")

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (375,378,379) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
del final_train_df['Unnamed: 0']

In [12]:
for c in [c for c in final_train_df.columns.tolist() if c not in final_kaggle_df.columns.tolist()]:
    del final_train_df[c]

In [13]:
assert final_train_df.shape[1] == final_kaggle_df.shape[1]

In [14]:
final_mega_df = pd.concat([final_train_df, final_kaggle_df], ignore_index=True)

In [15]:
assert final_mega_df.shape[1] == final_train_df.shape[1] == final_kaggle_df.shape[1]

In [16]:
assert final_mega_df.shape[0] == final_train_df.shape[0] + final_kaggle_df.shape[0]

## Data prep

### Notes
- ID_* columns should be embedded!
- _1line are no/yes columns, so they should be labeled...
- "ecology" has values of ['good', 'excellent', 'poor', 'satisfactory', 'no data'],... labelled/one-hot encoded/or embedded?
- church_count_500 and other _count values integer values - bucketize them?
- timestamp_day, timestamp_month, timestamp_year

In [17]:
class SberbankData():
    
    def __init__(self, csv_path, save_dir, use_cols=[], blacklist_cols=['id'], load_serializers_from_disk=False, prepare_dataset=True, predict_dataset=False):
        self.csv_path = csv_path
        self.save_dir = save_dir
        self.use_cols = use_cols
        self.blacklist_cols = blacklist_cols
        self.predict_dataset = predict_dataset
        
        self.load_dataset()
        
        if prepare_dataset:
            self.prepare_dataset(load_serializers_from_disk)
        
    def load_dataset(self, path=None):
        
        if type(path) == pd.core.frame.DataFrame:
            self.df = path
        else:
            if not path:
                path = self.csv_path
                if type(path) == pd.core.frame.DataFrame:
                    self.df = path
                else:
                    self.df = pd.read_csv(path)
            else:
                self.df = pd.read_csv(path)
        
    def _prepare_dataset(self, predict=False, trunc_fields=False):
        global LABEL
        t_fields = self.use_cols or ['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor','material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area', LABEL]
    
        if not predict:
            if trunc_fields:
                self.df = self.df[t_fields + LABEL]
            else:
                self.df = self.df
        else:
            if trunc_fields:
                self.df = self.df[t_fields]
            else:
                self.df = self.df
            
        fix_cols = [c for c in self.df.columns.tolist() if '+' in c]
        
        for c in fix_cols:
            self.df[c.replace("+","_")] = self.df[c]
            del self.df[c]
            
        
        self.df.fillna(0, inplace=True)
        self.df.replace('#!',0, inplace=True)
    
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['timestamp_day'] = self.df.apply(lambda row: row['timestamp'].day, axis=1)
        self.df['timestamp_month'] = self.df.apply(lambda row: row['timestamp'].month, axis=1)
        self.df['timestamp_year'] = self.df.apply(lambda row: row['timestamp'].year, axis=1)
        self.df['state'] = self.df.apply( lambda row: 5 if row['state'] == 33 else row['state'] , axis = 1).astype(int)
    
        
        self.df['material'] = self.df['material'].astype(int) 
        
        if 'Unnamed: 0' in self.df.columns.tolist():
            del self.df['Unnamed: 0']
            
    def get_bin_boundaries(self, df, col):
        step = int(df[col].std())
        if step == 0:
            return [float(df[col].min()), float(df[col].max())]
        else:
            return [float(g) for g in list(range( int(df[col].min()), int(df[col].max()), step))]
        
    def bucketize_column(self, df, col, bins=[]):
        if not bins:
            bins = self.get_bin_boundaries(df, col)
        return np.digitize(df[col].values, bins, right=True)
            
    def dedup(self, listy):
        return list(set(listy))
            
    def load_normalizers(self, force=False):
        if force:
            self.continuous_scalers = joblib.load(os.path.join(self.save_dir, 'normalizers', 'continuous_scalers.pkl'))
            self.label_scalers = joblib.load(os.path.join(self.save_dir, 'normalizers', 'label_scalers.pkl'))
            self.one_hot_scalers = joblib.load(os.path.join(self.save_dir, 'normalizers', 'onehot_scalers.pkl'))
        else:
            # Only load if there aren't already initialized.
            try:
                self.continuous_scalers
                self.label_scalers
                self.one_hot_scalers
            except AttributeError:
                # They don't exist, so let's load them.
                self.continuous_scalers = joblib.load(os.path.join(self.save_dir, 'normalizers', 'continuous_scalers.pkl'))
                self.label_scalers = joblib.load(os.path.join(self.save_dir, 'normalizers', 'label_scalers.pkl'))
                self.one_hot_scalers = joblib.load(os.path.join(self.save_dir, 'normalizers', 'onehot_scalers.pkl'))           
    
    def save_normalizers(self):
        joblib.dump(self.continuous_scalers, os.path.join(self.save_dir, 'normalizers', 'continuous_scalers.pkl'))
        joblib.dump(self.label_scalers, os.path.join(self.save_dir, 'normalizers', 'label_scalers.pkl'))
        joblib.dump(self.one_hot_scalers, os.path.join(self.save_dir, 'normalizers', 'onehot_scalers.pkl'))
    
    def _prepare_normalizers(self):
        # First, prepare the continuous columns scaler
        self.continuous_scalers = {}
        for col in self.all_continuous_columns:
            self.df[col] = [float(str(c).replace(",","")) for c in self.df[col].values.ravel().tolist()]
            if col in self.normalize_continuous_columns:
                scaler = preprocessing.StandardScaler()
                scaler.fit(self.df[col].values)
                self.continuous_scalers[col] = scaler
        
        # Second, prepare the label columns scaler
        self.label_scalers = {}
        for col in self.label_columns:
            scaler = preprocessing.LabelEncoder()
            scaler.fit(self.df[col].values.tolist() + [self._unknown_tag])
            self.label_scalers[col] = scaler
            
        # Before we create the one-hot scalers, we need to transform the bucketized features
        self._transform_bucketized_columns()
            
        # Third, prepare the one-hot columns scaler
        self.one_hot_scalers = {}
        for col in self.one_hot_columns:
            if col in self.bucketized_columns:
                num_bins = len(self.get_bin_boundaries(self.df, col)) + 1 # possible bucket ids are range(num_bins)
                oh_scaler = preprocessing.OneHotEncoder(n_values = num_bins, handle_unknown='ignore')
                oh_scaler.fit(np.array(self.df[col + self._bucketized_tag].values).reshape(-1, 1))
                self.one_hot_scalers[col] = (None, oh_scaler)
            else:
                l_scaler = preprocessing.LabelEncoder()
                out = l_scaler.fit_transform(self.df[col].values.tolist() + [self._unknown_tag])
                oh_scaler = preprocessing.OneHotEncoder(handle_unknown='ignore')
                oh_scaler.fit(np.array(out).reshape(-1, 1))
                self.one_hot_scalers[col] = (l_scaler, oh_scaler)
                
        self.save_normalizers()
    
    def _transform_bucketized_columns(self):
        for col in self.bucketized_columns:
            self.df[col + self._bucketized_tag] = self.bucketize_column(self.df, col)
            
    def _transform_continuous_columns(self):
        for col in self.all_continuous_columns:
            if col in self.normalize_continuous_columns:
                self.df[col + self._processed_tag] = self.continuous_scalers[col].transform([float(str(c).replace(",","")) for c in self.df[col].values.ravel().tolist()])
            else:
                # If we're not supposed to normalize this continuous column, then we simply keep the original values
                self.df[col + self._processed_tag] = [float(str(c).replace(",","")) for c in self.df[col].values.ravel().tolist()]
    
    
    def _clean_label_scaler_input(self, label_scaler, values):
        vals = []
        for val in values:
            if type(val) == str:
                if val in label_scaler.classes_:
                    vals.append(val)
                else:
                    vals.append(self._unknown_tag)
            elif type(val) == int:
                classes = [int(c) for c in label_scaler.classes_ if c != self._unknown_tag]
                if val in classes:
                    vals.append(val)
                else:
                    vals.append(self._unknown_tag)
        return vals
        
    def _transform_label_columns(self):
        for col in self.label_columns:
            l_scaler = self.label_scalers[col]
            # Replace unknown values with the unknown tag, in order to prevent problems with the label encoder
            # when it sees new values
            vals = self._clean_label_scaler_input(l_scaler, self.df[col].values.tolist())
            self.df[col + self._processed_tag] = l_scaler.transform(vals)
                
    def _transform_onehot_columns(self):
        for col in self.one_hot_columns:
            if col in self.bucketized_columns:
                # This is a bucketized column
                oh_scaler = self.one_hot_scalers[col][1]
                # We've already bucketized this column by this time,
                # so all we have to do now is to transform these bucketized values to a one-hot encoding
                self.df[col + self._bucketized_tag] = oh_scaler.transform(self.df[col + self._bucketized_tag].values.reshape(-1, 1)).toarray().tolist()
            else:
                # This is an ordinary one-hot column
                l_scaler, oh_scaler = self.one_hot_scalers[col]
                vals = self._clean_label_scaler_input(l_scaler, self.df[col].values.tolist())
                out = l_scaler.transform(vals)
                self.df[col + self._processed_tag] = oh_scaler.transform(np.array(out).reshape(-1, 1)).toarray().tolist()
                
    
    def transform_dataset(self):
        self._transform_bucketized_columns()     
        self._transform_continuous_columns()
        self._transform_label_columns()
        self._transform_onehot_columns()
        
    def _extract_processed_feature(self, df, feature_col):
        feature_col = feature_col + self._processed_tag
        vals = np.expand_dims(df[feature_col].values, axis=1)
        return vals
        
    def extract_training_data(self, df = None, predict=False):
        # Create an x_train, y_train pair of numpy arrays, that can be feed into a machine learning model (eg. deep learning, etc.)
        global LABEL
        
        inputs = []
        embed_inputs = {}
        
        if isinstance(df, type(None)):
            df = self.df
        
        for col in self.all_feature_columns:
            
            if col not in self.embedding_columns:
                # This isn't an embedding column
                val = np.expand_dims(self._extract_processed_feature(df, col), axis=1)
                inputs.append(val)
            
                if col in self.bucketized_columns:
                    # If this column is also a bucketized column, also add the bucketized feature to this input list.
                    val = np.expand_dims(self._extract_processed_feature(df, col + "_bucketized"), axis=1)
                    inputs.append(val)
            else:
                # This is an embedding column
                val = np.expand_dims(self._extract_processed_feature(df, col), axis=1)
                embed_inputs[col] = val
        
        # Combine all inputs
        inputs = np.hstack(tuple(inputs))
        
        if predict:
            return inputs, embed_inputs
        
        outputs = np.expand_dims(df[LABEL].values, axis=1)
        return (inputs, embed_inputs), outputs
            
        
    def make_train_validate_test_split(self, train_percent=.6, validate_percent=.2, seed=None, transform=False):
        
        if transform:
            self.transform_dataset()
            
        np.random.seed(seed)
        perm = np.random.permutation(self.df.index)
        m = len(self.df)
        train_end = int(train_percent * m)
        validate_end = int(validate_percent * m) + train_end
        train = self.df.ix[perm[:train_end]]
        validate = self.df.ix[perm[train_end:validate_end]]
        test = self.df.ix[perm[validate_end:]]
        return train, validate, test
    
            
    def _prepare_feature_columns(self, load_serializers_from_disk=False):
        """
        Notes
        ID_* columns should be embedded!
        _1line are no/yes columns, so they should be labeled...
        "ecology" has values of ['good', 'excellent', 'poor', 'satisfactory', 'no data'],... labelled/one-hot encoded/or embedded?
        church_count_500 and other _count values integer values - bucketize them?
        timestamp_day, timestamp_month, timestamp_year
        """

        self._id_tag = "ID_"
        self._1line_tag = "_1line"
        self._raion_tag = "_raion"
        self._count_tag = "_count"
        self._km_tag = "_km"
        self._sq_tag = "_sq"
        
        self._unknown_tag = "_unknown_tag"
        self._processed_tag = "_processed_feature"
        self._bucketized_tag = "_bucketized" + self._processed_tag
        
        self.all_feature_columns = self.dedup([c for c in self.df.columns.tolist() if c not in self.blacklist_cols and self._processed_tag not in c])

        def get_subcolumns_by_tag(tag):
            fts = []
            for f in self.all_feature_columns:
                if tag in f:
                    fts.append(f)
            return fts

        def get_raion_columns(return_yesno=True, return_cont=False):
            yesno_cols = []
            continuous_cols = []
            for c in get_subcolumns_by_tag(self._raion_tag):
                if any(w in self.df[c].unique().tolist() for w in ('yes', 'no')):
                    yesno_cols.append(c)
                else:
                    continuous_cols.append(c)
            if return_yesno:
                return yesno_cols
            elif return_cont:
                return continuous_cols
            else:
                return yesno_cols, continuous_cols
    

        self.integerized_embedding_columns = get_subcolumns_by_tag(self._id_tag)
        self.unlabeled_embedding_columns = ["ecology", "sub_area"]

        self.embedding_columns = self.integerized_embedding_columns + self.unlabeled_embedding_columns

        # Note: This will not modify the existing feature column(s).
        # Rather, it will create a new supplemental feature column with bucketization
        # Also, bucketized columns will be one-hot encoded, after being label encoded...
        self.bucketized_columns = [
            # Specific columns to bucketize...
        ] + get_subcolumns_by_tag(self._count_tag) + get_subcolumns_by_tag(self._km_tag) + get_subcolumns_by_tag(self._sq_tag)

        self.label_columns = [
            # Specific label columns to include...    
        ] + get_raion_columns(return_yesno=True) + get_subcolumns_by_tag(self._1line_tag) + self.embedding_columns
        
        # One-hot columns that need to be label encoded
        self.one_hot_columns_to_be_labeled = [
            'material',
            'state',
            'product_type',
            'culture_objects_top_25',
        ]
        
        self.one_hot_columns = self.one_hot_columns_to_be_labeled  + self.bucketized_columns
        
        # Don't normalize one-hot, label, and integerized embedding columns, although they contain numbers...
        self.exclude_normalize_continuous_columns = [
            # Specific columns to exclude from normalization...
            # TODO: Should we exclude timestamp_month, day, and year??
        ] + self.one_hot_columns + self.label_columns + self.integerized_embedding_columns
        
        self.normalize_continuous_columns = [c for c in self.all_feature_columns if c not in self.exclude_normalize_continuous_columns]
        self.all_continuous_columns = [c for c in self.all_feature_columns if c not in self.one_hot_columns_to_be_labeled + self.label_columns + self.embedding_columns]
        
        for int_emb_col in self.integerized_embedding_columns:
            self.df[int_emb_col] = self.df[int_emb_col].astype(int) # Convert all integer id embedding columns to int
            # as, some may be a float...
        
        # Only prepare the normalizers if this is a main training set (and NOT a predict set...)
        if not self.predict_dataset:
            if load_serializers_from_disk:
                self.load_normalizers(force=True)
            else:
                self._prepare_normalizers()
        else:
            self.load_normalizers(force=load_serializers_from_disk)
        
        
    def prepare_dataset(self, load_serializers_from_disk):
        self._prepare_dataset()
        self._prepare_feature_columns(load_serializers_from_disk) 

In [18]:
def determine_dimensions(num_unique, r = 0, k = 1):
    if r == 0:
        return int(math.log(num_unique, 2))
    else:
        return k * int(num_unique ** 1./4.)

In [19]:
sbankdata = SberbankData(final_mega_df, '/mnt/h/Kaggle/Competitions/Russian Bank/code/data')

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

In [20]:
sbankdata.load_dataset(final_train_df)

In [ ]:
sbankdata.prepare_dataset(True)

In [ ]:
sbankdata.transform_dataset()

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

In [27]:
train_set, val_set, test_set = sbankdata.make_train_validate_test_split()

In [28]:
# For the purposes of Keras, let's join the train and val set
train_set = final_mega_df = pd.concat([train_set, val_set], ignore_index=True)

In [44]:
((train_inputs, train_embeddings_input), train_outputs) = sbankdata.extract_training_data(train_set)

In [51]:
train_inputs.shape, train_outputs.shape

((24376, 569), (24376, 1))

In [46]:
train_inputs = np.squeeze(train_inputs, axis=2)

In [47]:
((test_inputs, test_embeddings_input), test_outputs) = sbankdata.extract_training_data(test_set)

In [52]:
test_inputs.shape, test_outputs.shape

((6095, 569), (6095, 1))

In [49]:
test_inputs = np.squeeze(test_inputs, axis=2)

In [50]:
np.unique(train_embeddings_input['sub_area']).size

145

## Model

In [53]:
from keras.layers import Dense, Input, Dropout, Concatenate, Embedding, Flatten
from keras.models import Model
from keras import backend as K
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [54]:
SPARE_CAPACITY = 100

In [55]:
def prepare_embeddings_header(embeddings_input_names, label_scalers):
    global SPARE_CAPACITY
    header = {}
    for e_name in embeddings_input_names:
        vocab_size = label_scalers[e_name].classes_.size + SPARE_CAPACITY
        dimensions = determine_dimensions(vocab_size)
        header[e_name] = (vocab_size, dimensions)
    return header

In [56]:
def model_v2_1(num_continuous_inputs, embeddings_header):
    
    """
    embeddings_header is a dict mapping from embedding input name -> (vocab_size, embedding)
    """
    
    with tf.name_scope("realty_housing_price_model"):
        with tf.name_scope("inputs"):
            continuous_input = Input(shape=(num_continuous_inputs,), name="continuous_input")
            
            embedding_inputs = []
            embedding_container = {}
            
            for embedding_input_name, (vocab_size, embedding) in embeddings_header.items():
                e_input = Input(shape=(1,), name=embedding_input_name)
                embedding_inputs.append(e_input)
                embedding_container[embedding_input_name] = (e_input, vocab_size, embedding)
            
        with tf.name_scope("embeddings"):
            embeddings = []
            for e_input_name, (e_input, vocab_size, embedding) in embedding_container.items():
                emb = Embedding(output_dim=embedding, input_dim=vocab_size, input_length=1, name=e_input_name + "_embedding")(e_input)
                emb = Flatten()(emb)
                embeddings.append(emb)
                
        with tf.name_scope("merge_layer"):
            merged_input_vector = Concatenate()([continuous_input] + embeddings)
            
        with tf.name_scope("fully_connected"):
            x = Dense(512, activation='relu')(merged_input_vector)
            x = Dropout(0.4)(x)
            x = Dense(512, activation='relu')(x)
            x = Dropout(0.4)(x)
            x = Dense(256, activation='relu')(x)
            x = Dropout(0.25)(x)
            output = Dense(1, activation='relu', name='output')(x)
            
        return [continuous_input] + embedding_inputs, output            

In [57]:
embeddings_header = prepare_embeddings_header(train_embeddings_input.keys(), sbankdata.label_scalers)
num_continuous_inputs = train_inputs.shape[1]

In [58]:
embeddings_header

{'ID_big_road1': (149, 7),
 'ID_big_road2': (159, 7),
 'ID_bus_terminal': (115, 6),
 'ID_metro': (325, 8),
 'ID_railroad_station_avto': (236, 7),
 'ID_railroad_station_walk': (237, 7),
 'ID_railroad_terminal': (109, 6),
 'ecology': (106, 6),
 'sub_area': (247, 7)}

In [59]:
inputs, outputs = model_v2_1(num_continuous_inputs=num_continuous_inputs, embeddings_header=embeddings_header)
realty_price_model = Model(inputs=inputs, outputs=outputs)

In [60]:
def root_mean_squared_logarithmic_error(y_true, y_pred):
    y_pred_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    y_true_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    return K.sqrt(K.mean(K.square(y_pred_log - y_true_log), axis = -1))

In [61]:
realty_price_model.compile(optimizer = "adam", loss = root_mean_squared_logarithmic_error)

In [293]:
plot_model(realty_price_model, to_file='/mnt/h/Kaggle/Competitions/Russian Bank/models/' + version_str + '/model_' + version_str + '.png')

OSError: [Errno 12] Cannot allocate memory

In [ ]:
SVG(model_to_dot(realty_price_model).create(prog='dot', format='svg'))

## Train the model

In [62]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau

In [63]:
model_checkpoint = ModelCheckpoint('/mnt/h/Kaggle/Competitions/Russian Bank/models/' + version_str + '/keras_model', monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss',  patience=20, verbose=0, mode='auto')
tensorboard = TensorBoard(log_dir='/mnt/h/Kaggle/Competitions/Russian Bank/models/' + version_str + '/keras_model/logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=5, embeddings_layer_names=None, embeddings_metadata=None)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=5, min_lr=0.001) 
callbacks = [model_checkpoint,early_stopping, tensorboard, reduce_lr]

In [64]:
epochs = 100
val_split = 0.2

In [79]:
train_inputs.shape, train_outputs.shape

((24376, 569), (24376, 1))

In [88]:
train_inputs[0].ravel()

array([2.0, [0.0, 1.0, 0.0, 0.0, 0.0, 0.0], -0.5360235995900599, 0.0,
       [1.0, 0.0, 0.0], 2.0, [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], 17000.0,
       [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1, 0.0,
       [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0,
       [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       -0.8983141417756757, 234.0, [0.0, 0.0, 0.0, 0.0], 0.632322735360394,
       3.187880601, [0.0, 1.0, 0.0], 0.0,
       [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       21.18120518, [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       13.33260292,
       [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       0.06658913241278343, 8.076809259,
       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       13.010854,
       [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0,
       [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       0.0, [1.0, 0.0

In [74]:
np.squeeze(train_embeddings_input['ID_railroad_station_avto'], axis=2)

array([[20],
       [64],
       [62],
       ..., 
       [57],
       [62],
       [ 8]])

### Fix embedding inputs...

In [75]:
for k in train_embeddings_input:
    train_embeddings_input[k] = np.squeeze(train_embeddings_input[k], axis=2)

In [76]:
for k in test_embeddings_input:
    test_embeddings_input[k] = np.squeeze(test_embeddings_input[k], axis=2)

## Train model

In [78]:
realty_price_model.fit(x=dict({'continuous_input': train_inputs}.items() + train_embeddings_input.items()),
          y=train_outputs,
          validation_split=val_split,
          epochs=epochs, callbacks = callbacks)

Train on 19500 samples, validate on 4876 samples
Epoch 1/100


ValueError: setting an array element with a sequence.

## Kaggle submission

In [326]:
kaggle_test = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/test.csv/test.csv")

In [327]:
kaggle_test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [328]:
macro_df = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/macro.csv/macro.csv")

In [329]:
final_kaggle_df = pd.merge(kaggle_test, macro_df, on='timestamp')

In [330]:
final_kaggle_df = prepare_dataset(final_kaggle_df)

In [331]:
final_kaggle_df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,timestamp_converted,timestamp_day,timestamp_month,timestamp_year,state_refractored
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,3
1,30475,2015-07-01,79.2,0.0,8,17,1,0.0,3,1.0,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,2
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,3
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,1


In [332]:
final_kaggle_df = final_kaggle_df[all_feature_columns ]

In [231]:
[c for c in final_kaggle_df.columns.tolist() if "normalized" in c]

['sport_count_5000_bucketized_normalized_feature',
 'mosque_count_500_bucketized_normalized_feature',
 'cafe_count_5000_price_2500_bucketized_normalized_feature',
 'build_count_brick_bucketized_normalized_feature',
 'office_count_1500_bucketized_normalized_feature',
 'office_count_500_bucketized_normalized_feature',
 'cafe_count_1000_price_1000_bucketized_normalized_feature',
 'build_count_foam_bucketized_normalized_feature',
 'cafe_count_3000_price_1000_bucketized_normalized_feature',
 'leisure_count_1000_bucketized_normalized_feature',
 'cafe_count_1500_price_1500_bucketized_normalized_feature',
 'mosque_count_2000_bucketized_normalized_feature',
 'trc_count_500_bucketized_normalized_feature',
 'leisure_count_2000_bucketized_normalized_feature',
 'trc_count_2000_bucketized_normalized_feature',
 'cafe_count_500_price_1500_bucketized_normalized_feature',
 'cafe_count_3000_price_1500_bucketized_normalized_feature',
 'build_count_after_1995_bucketized_normalized_feature',
 'cafe_count_50

In [333]:
final_kaggle_df['old_education_build_share'] = final_kaggle_df.apply(lambda row: int(str(row['old_education_build_share']).replace(",","")), axis=1)

In [334]:
final_kaggle_df['modern_education_share'] = final_kaggle_df.apply(lambda row: int(str(row['modern_education_share']).replace(",","")), axis=1)

In [335]:
default_product_type = "Investment"

In [336]:
final_kaggle_df['product_type'] = final_kaggle_df.apply(lambda row: row['product_type'] if row['product_type'] != 0 else default_product_type, axis=1)

In [337]:
final_kaggle_df = transform_dataset(final_kaggle_df)

In [198]:
final_kaggle_df[[c for c in final_kaggle_df.columns.tolist() if "normalized" in c]]

,sport_count_5000_bucketized_normalized_feature,mosque_count_500_bucketized_normalized_feature,cafe_count_5000_price_2500_bucketized_normalized_feature,build_count_brick_bucketized_normalized_feature,office_count_1500_bucketized_normalized_feature,office_count_500_bucketized_normalized_feature,cafe_count_1000_price_1000_bucketized_normalized_feature,build_count_foam_bucketized_normalized_feature,cafe_count_3000_price_1000_bucketized_normalized_feature,leisure_count_1000_bucketized_normalized_feature,...,nuclear_reactor_raion_normalized_feature,railroad_terminal_raion_normalized_feature,incineration_raion_normalized_feature,big_market_raion_normalized_feature,thermal_power_plant_raion_normalized_feature,big_road1_1line_normalized_feature,water_1line_normalized_feature,railroad_1line_normalized_feature,ecology_normalized_feature,sub_area_normalized_feature
0,1,0,0,2,0,0,0,1,1,0,...,1,1,1,1,1,1,1,1,5,38
1,1,0,1,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,3,103
2,2,0,1,2,1,0,1,0,1,1,...,1,1,1,1,2,1,1,1,4,84
3,1,0,1,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,3,105
4,1,0,1,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,3,103
5,3,0,2,2,1,1,1,0,1,0,...,1,1,1,1,1,1,1,2,5,24
6,1,0,1,1,1,0,1,0,1,1,...,1,1,1,1,1,1,1,1,2,124
7,1,0,1,0,0,0,0,0,1,0,...,1,1,1,2,1,1,1,1,3,102
8,1,0,1,2,0,0,0,0,1,0,...,1,1,1,1,1,1,1,1,3,136
9,1,0,1,0,0,0,0,0,1,0,...,1,1,1,1,1,1,1,1,3,103


In [199]:
[c for c in final_kaggle_df.columns.tolist() if "normalized" in c]

['sport_count_5000_bucketized_normalized_feature',
 'mosque_count_500_bucketized_normalized_feature',
 'cafe_count_5000_price_2500_bucketized_normalized_feature',
 'build_count_brick_bucketized_normalized_feature',
 'office_count_1500_bucketized_normalized_feature',
 'office_count_500_bucketized_normalized_feature',
 'cafe_count_1000_price_1000_bucketized_normalized_feature',
 'build_count_foam_bucketized_normalized_feature',
 'cafe_count_3000_price_1000_bucketized_normalized_feature',
 'leisure_count_1000_bucketized_normalized_feature',
 'cafe_count_1500_price_1500_bucketized_normalized_feature',
 'mosque_count_2000_bucketized_normalized_feature',
 'trc_count_500_bucketized_normalized_feature',
 'leisure_count_2000_bucketized_normalized_feature',
 'trc_count_2000_bucketized_normalized_feature',
 'cafe_count_500_price_1500_bucketized_normalized_feature',
 'cafe_count_3000_price_1500_bucketized_normalized_feature',
 'build_count_after_1995_bucketized_normalized_feature',
 'cafe_count_50

In [338]:
extract_normalized_feature(final_kaggle_df, "sub_area")

array([ 38, 103,  84, ..., 134,  79,  19])

In [339]:
kaggle_split_df_continuous_features = extract_normalized_features(final_kaggle_df)
kaggle_split_df_categorical_features = extract_categorical_features(final_kaggle_df, include_embedding_cols=False)
kaggle_split_df_embedding_features = extract_embedding_features(final_kaggle_df)
    
kaggle_split_df_embedding_features_final = {}
for k, v in kaggle_split_df_embedding_features.items():
    kaggle_split_df_embedding_features_final[k + "_input"] = v
       
kaggle_split_df_combined_inputs = combine_inputs([kaggle_split_df_continuous_features] 
                                                     +  kaggle_split_df_categorical_features.values())

In [340]:
kaggle_split_df_categorical_features

{'big_market_raion': array([[1],
        [1],
        [1],
        ..., 
        [1],
        [2],
        [1]]), 'big_road1_1line': array([[1],
        [1],
        [1],
        ..., 
        [1],
        [1],
        [1]]), 'culture_objects_top_25': array([[ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        ..., 
        [ 0.,  1.],
        [ 1.,  0.],
        [ 1.,  0.]]), 'detention_facility_raion': array([[1],
        [1],
        [1],
        ..., 
        [2],
        [1],
        [1]]), 'incineration_raion': array([[1],
        [1],
        [1],
        ..., 
        [1],
        [1],
        [1]]), 'material': array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.]]), 'nuclear_reactor_raion': array([[1],
        [1],
        [1],
        ..., 
   

In [341]:
kaggle_split_df_embedding_features

{'ID_big_road1': array([[ 2],
        [13],
        [10],
        ..., 
        [ 4],
        [23],
        [ 2]]), 'ID_big_road2': array([[38],
        [27],
        [ 1],
        ..., 
        [34],
        [52],
        [40]]), 'ID_bus_terminal': array([[ 9],
        [ 8],
        [ 3],
        ..., 
        [13],
        [ 6],
        [ 2]]), 'ID_metro': array([[ 45],
        [ 21],
        [ 44],
        ..., 
        [120],
        [ 53],
        [142]]), 'ID_railroad_station_avto': array([[39],
        [24],
        [59],
        ..., 
        [32],
        [27],
        [21]]), 'ID_railroad_station_walk': array([[ 39.],
        [ 24.],
        [ 68.],
        ..., 
        [  5.],
        [ 27.],
        [ 82.]]), 'ID_railroad_terminal': array([[32],
        [50],
        [ 5],
        ..., 
        [32],
        [32],
        [32]]), 'ecology': array([[5],
        [3],
        [4],
        ..., 
        [1],
        [4],
        [4]]), 'sub_area': array([[ 38],
        [103],


In [342]:
kaggle_split_df_continuous_features

array([[-0.838086  ,  0.03928507, -0.07009939, ...,  1.89720164,
        -0.48593907, -1.83239346],
       [-0.8812901 ,  0.16129706, -0.07009939, ...,  1.89720164,
        -0.49387983, -1.83239346],
       [ 0.39323092, -0.14841033, -0.07009939, ...,  1.89720164,
        -0.10478255, -1.83239346],
       ..., 
       [ 3.24470169, -0.19664223, -0.07009939, ...,  0.24523479,
         4.54850328, -1.83239346],
       [-0.21162651,  0.01626024, -0.07009939, ...,  0.24523479,
        -0.36682766, -1.83239346],
       [-0.23322857,  0.2272611 , -0.07009939, ...,  0.24523479,
        -0.20007168, -1.83239346]])

In [343]:
vs = []
for column in one_hot_columns:
    vals = final_kaggle_df[column].values.tolist()
    vals = [c if c in known_values[column] else unknown_label for c in vals]
    if unknown_label in vals:
        vs.append((column, vals))

In [344]:
[v[0] for v in vs]

[]

In [345]:
kaggle_preds = real_estate_price_model.predict(x=dict({
                    'main_input' : kaggle_split_df_combined_inputs
                }.items() + kaggle_split_df_embedding_features_final.items()))

In [346]:
kaggle_preds

array([[  3.07068355e+10],
       [  3.07081196e+10],
       [  3.07076506e+10],
       ..., 
       [  9.80176000e+06],
       [  1.45478860e+07],
       [  1.71274320e+07]], dtype=float32)

In [191]:
final_kaggle_df['price_doc'] = kaggle_preds

In [192]:
final_kaggle_df['price_doc']

0       4.093042e+10
1       4.093284e+10
2       4.093333e+10
3       4.093235e+10
4       4.092968e+10
5       4.093546e+10
6       4.093052e+10
7       4.092911e+10
8       4.093080e+10
9       4.093024e+10
10      4.093242e+10
11      4.092887e+10
12      4.092848e+10
13      4.092789e+10
14      4.093016e+10
15      4.093108e+10
16      4.093927e+10
17      4.093849e+10
18      4.093026e+10
19      4.093700e+10
20      4.093147e+10
21      4.093658e+10
22      4.093368e+10
23      4.093388e+10
24      4.093059e+10
25      4.093547e+10
26      4.093610e+10
27      4.093445e+10
28      4.092772e+10
29      4.093438e+10
            ...     
7632    1.344982e+07
7633    8.391098e+06
7634    1.654414e+07
7635    1.180624e+07
7636    6.630701e+06
7637    9.822717e+06
7638    7.509546e+06
7639    1.351762e+07
7640    1.002495e+07
7641    7.301856e+06
7642    1.362228e+07
7643    1.310399e+07
7644    1.443113e+07
7645    7.393820e+06
7646    1.267712e+07
7647    1.206970e+07
7648    1.271

In [193]:
4.093042e10

40930420000.0

In [271]:
final_kaggle_df[['id', 'price_doc']]

,id,price_doc
0,30474,5880385.00
1,30475,8747043.00
2,30476,6298690.00
3,30477,6858484.00
4,30478,5549552.00
5,30479,8727317.00
6,30480,4832307.00
7,30481,4594226.00
8,30482,5736204.00
9,30483,5148752.00


In [272]:
final_kaggle_df[['id', 'price_doc']].to_csv("/mnt/h/Kaggle/Competitions/Russian Bank/models/v" + str(version) + "/submission.csv", index=False) # output submission csv file